CC5213 - Recuperación de Información Multimedia
==
Profesor: Juan Manuel Barrios  
Fecha: 26 abril 2021

Búsquedas aproximadas con PCA
==

### Leer los datos

In [3]:
import os
import numpy

def load_file(filename, dimensions):
    assert os.path.isfile(filename), "no existe archivo " + filename
    mat = numpy.fromfile(filename, dtype=numpy.float32)
    return numpy.reshape(mat, (-1, dimensions))

class Dataset:   
    def __init__(self, nombre, archivo_q, archivo_r, dimensiones):
        self.nombre = nombre
        self.q = load_file(archivo_q, dimensiones)
        self.r = load_file(archivo_r, dimensiones)
        print("Dataset {}: Q={} R={}".format(self.nombre, self.q.shape, self.r.shape))

dataset1 = Dataset("SIFT", "SIFT-Q-128_float32.bin", "SIFT-R-128_float32.bin", 128)

dataset2 = Dataset("MEL", "MEL-Q-128_float32.bin", "MEL-R-128_float32.bin", 128)

AssertionError: no existe archivo SIFT-Q-128_float32.bin

### Objeto que hace la comparación

In [1]:
import time
import pyflann

class EvaluarPCA():   
    def __init__(self, dataset):
        self.dataset = dataset
        self.flann = pyflann.FLANN()
        self.dimensiones = []
        self.varianzas = []
        self.precisiones = []
        self.eficiencias = []

    def linear_scan(self):
        print("{} linear scan de Q={}x R={}".format(
            self.dataset.nombre, self.dataset.q.shape, self.dataset.r.shape))
        self.flann.build_index(self.dataset.r, algorithm="linear")
        t0 = time.time()
        self.gt_nns, self.gt_dists = self.flann.nn_index(self.dataset.q, num_neighbors=1, cores=2)
        self.gt_segundos = time.time() - t0
        print("{} linear scan = {:.1f} segundos".format(self.dataset.nombre, self.gt_segundos))

    def calcular_PCA(self):
        self.promedios = self.dataset.r.mean(axis=0)
        datos_centrados = self.dataset.r - self.promedios
        #matriz de covarianza
        matriz_covarianza = numpy.cov(datos_centrados.transpose(), bias=True)
        #valores y vectores propios de la matriz de covarianza
        eigenvalues, eigenvectors = numpy.linalg.eig(matriz_covarianza)
        #indices ordenados
        indices_menor_a_mayor = eigenvalues.argsort()
        indices_mayor_a_menor = indices_menor_a_mayor[::-1]
        #guardar valores y vectores propios de mayor a menor
        self.eigenvalues = eigenvalues[indices_mayor_a_menor]
        self.eigenvectors = eigenvectors[:,indices_mayor_a_menor]

    def evaluar_busqueda(self, nns, dists, tiempo):
        correctas = 0
        incorrectas = 0
        for i in range(len(self.gt_nns)):
            if nns[i] == self.gt_nns[i]: 
                correctas += 1
            else:
                incorrectas += 1 
        precision = 100 * correctas / (correctas + incorrectas)
        eficiencia = 100 * tiempo / self.gt_segundos
        return precision, eficiencia

    def reducir_y_buscar(self, new_dims):
        varianza_retenida = numpy.sum(self.eigenvalues[:new_dims]) / numpy.sum(self.eigenvalues) * 100
        dimension = new_dims / self.dataset.r.shape[1] * 100
        #reducir R (offline)
        r_centrado = self.dataset.r - self.promedios
        transformacion = self.eigenvectors[:, :new_dims]
        r_newdim = r_centrado.dot(transformacion)
        #reducir Q (online)
        q_centrado = self.dataset.q - self.promedios
        q_newdim = q_centrado.dot(transformacion)
        #buscar entre Q y R reducidos con linear scan
        self.flann.build_index(r_newdim, algorithm="linear")
        t0 = time.time()
        nns_search, dists_search = self.flann.nn_index(q_newdim, num_neighbors=1, cores=2)
        segundos = time.time() - t0
        #medir resultado de la busqueda
        precision, eficiencia = self.evaluar_busqueda(nns_search, dists_search, segundos)
        #guardar resultados
        print("{} PCA-{} dim={:.1f}%\tvar={:.1f}%\tprecision={:.1f}%\ttiempo={:.1f}%\tsegundos={:.1f}".format(self.dataset.nombre, new_dims, dimension, varianza_retenida, precision, eficiencia, segundos))
        self.dimensiones.append(dimension)
        self.varianzas.append(varianza_retenida)
        self.precisiones.append(precision)
        self.eficiencias.append(eficiencia)

In [2]:
ev1 = EvaluarPCA(dataset1)
ev1.linear_scan()
ev1.calcular_PCA()
for dims in 2, 4, 8, 16, 24, 32, 48, 64, 80, 96, 112, 128:
    ev1.reducir_y_buscar(dims)

ev2 = EvaluarPCA(dataset2)
ev2.linear_scan()
ev2.calcular_PCA()
for dims in 2, 4, 8, 16, 24, 32, 48, 64, 80, 96, 112, 128:
    ev2.reducir_y_buscar(dims)

NameError: name 'dataset1' is not defined

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib widget

plt.plot(ev1.precisiones, ev1.eficiencias, label=ev1.dataset.nombre, color='r', linestyle='--', marker='o', markerfacecolor='c', markersize=8)
plt.plot(ev2.precisiones, ev2.eficiencias, label=ev2.dataset.nombre, color='g', linestyle='-.', marker='^', markerfacecolor='m', markersize=8)
plt.xlabel('Precisión NN %')
plt.ylabel('Tiempos %')
plt.xlim(0,100)
plt.ylim(0,100)
plt.legend()
plt.show()